In [1]:
import pandas as pd
from fastparquet import write
# 댓글 크롤링 데이터 불러옴
df=pd.read_parquet('comment_id.parquet',engine='fastparquet')

In [2]:
# 댓글 아이디 별로 분류, 웹툰 댓글 단 개수 값 넣어서
df = df.groupby(['아이디','제목']).size().reset_index(name='count')
df = df.pivot('아이디', '제목', 'count').fillna(0) 
print(df)
df.to_parquet('trying.parquet', compression='gzip')
#print(df)

제목             0.0MHz  0.1초의 설렘  100%의 그녀  11년 후 우리는  12단곡괭이  12월   12차원 소년들  \
아이디                                                                            
       불렉션        0.0       0.0       0.0        0.0     0.0   0.0       0.0   
       찐          0.0       0.0       0.0        0.0     0.0   0.0       0.0   
 Lr매ㄱr고ㅍr         0.0       0.0       0.0        0.0     0.0   0.0       0.0   
 chahy            0.0       0.0       0.0        0.0     0.0   0.0       0.0   
 곰지123            0.0       0.0       0.0        0.0     0.0   0.0       0.0   
...               ...       ...       ...        ...     ...   ...       ...   
＊ㄴδㅁξ - - ª..     0.0       0.0       0.0        0.0     0.0   0.0       0.0   
＊오가＊              0.0       0.0       0.0        0.0     0.0   0.0       0.0   
＊좋은생각＊            0.0       0.0       0.0        0.0     0.0   0.0       0.0   
￡0ⓥe ⓘs...        0.0       0.0       0.0        0.0     0.0   0.0       0.0   
￡ΟνΕ　 Ιε 。。♡      0.0       0.0       0.

In [3]:
# 비슷한 사용자 k 명 찾기
from sklearn.metrics.pairwise import cosine_similarity
import operator

def similar_users(user_id, matrix, k=5):
    # create a df of just the current user
    user = matrix[matrix.index == user_id]
    
    # and a df of all other users
    other_users = matrix[matrix.index != user_id]
    
    # calc cosine similarity between user and each other user
    similarities = cosine_similarity(user,other_users)[0].tolist()
    
    # create list of indices of these users
    indices = other_users.index.tolist()
    
    # create key/values pairs of user index and their similarity
    index_similarity = dict(zip(indices, similarities))
    
    # sort by similarity
    index_similarity_sorted = sorted(index_similarity.items(), key=operator.itemgetter(1))
    index_similarity_sorted.reverse()
    
    # grab k users off the top
    top_users_similarities = index_similarity_sorted[:k]
    users = [u[0] for u in top_users_similarities]
    
    return users
    


In [4]:
#비슷한 사용자들 평균내서, 추천 웹툰 목록 만듬
def recommend_item(user_index, similar_user_indices, matrix):
    
    # load vectors for similar users
    similar_users = matrix[matrix.index.isin(similar_user_indices)]
    # calc avg ratings across the 3 similar users
    similar_users = similar_users.mean(axis=0)
    # convert to dataframe so its easy to sort and filter
    similar_users_df = pd.DataFrame(similar_users, columns=['mean'])
    
    # load vector for the current user
    user_df = matrix[matrix.index == user_index]
    # transpose it so its easier to filter
    user_df_transposed = user_df.transpose()
    # rename the column as 'rating'
    user_df_transposed.columns = ['rating']
    # remove any rows without a 0 value. webtoon not watched yet
    user_df_transposed = user_df_transposed[user_df_transposed['rating']==0]
    # generate a list of webtoons the user has not seen
    webtoon_unseen = user_df_transposed.index.tolist()
    
    # filter avg ratings of similar users for only webtoon the current user has not seen
    similar_users_df_filtered = similar_users_df[similar_users_df.index.isin(webtoon_unseen)]
    # order the dataframe
    similar_users_df_ordered = similar_users_df_filtered.sort_values(by=['mean'], ascending=False)
    print(similar_users_df_ordered)
    
    return similar_users_df_ordered #items



In [5]:
def recommend_by_user(user_webtoon_input) :
    # 전체 데이터 프레임에 최근 본 웹툰 3개의 가중치값을 1로 두어 행을 추가
    
    df=pd.read_parquet('trying.parquet',engine='fastparquet')
    df.loc['user_input']={user_webtoon_input[0]:1,user_webtoon_input[1]:1,user_webtoon_input[2]:1}
    df=df.fillna(0)
    
    current_user = "user_input"
    # try it out
    similar_user_indices = similar_users(current_user, df)
    print(similar_user_indices)

    recommend_list  = recommend_item(current_user, similar_user_indices, df)
    return recommend_list 
    

In [6]:
# 사용자가 최근 본 웹툰 3개 여기에 입력
user_webtoon_input=['놓정동화','슈퍼 시크릿','삼국지톡']

recommend_list= recommend_by_user(user_webtoon_input)

['wo30****', 'sr09****', 'rbdh****', 'kwui****', 'ose0****']
            mean
제목              
일상날개짓        0.6
환생동물학교       0.2
하이브 3        0.2
놓지마 정신줄      0.2
밥 먹고 갈래...   0.2
...          ...
마왕비 납시오      0.0
마야비          0.0
마애(磨崖)       0.0
마스크걸         0.0
힙한남자         0.0

[1819 rows x 1 columns]


In [7]:
print(recommend_list)

            mean
제목              
일상날개짓        0.6
환생동물학교       0.2
하이브 3        0.2
놓지마 정신줄      0.2
밥 먹고 갈래...   0.2
...          ...
마왕비 납시오      0.0
마야비          0.0
마애(磨崖)       0.0
마스크걸         0.0
힙한남자         0.0

[1819 rows x 1 columns]


In [8]:
recommend_list.head(10)

,mean
제목,
일상날개짓,0.6
환생동물학교,0.2
하이브 3,0.2
놓지마 정신줄,0.2
밥 먹고 갈래...,0.2
부부생활,0.2
열정호구,0.2
2015 소름,0.2
0.0MHz,0.0


In [9]:
print(df)

제목             0.0MHz  0.1초의 설렘  100%의 그녀  11년 후 우리는  12단곡괭이  12월   12차원 소년들  \
아이디                                                                            
       불렉션        0.0       0.0       0.0        0.0     0.0   0.0       0.0   
       찐          0.0       0.0       0.0        0.0     0.0   0.0       0.0   
 Lr매ㄱr고ㅍr         0.0       0.0       0.0        0.0     0.0   0.0       0.0   
 chahy            0.0       0.0       0.0        0.0     0.0   0.0       0.0   
 곰지123            0.0       0.0       0.0        0.0     0.0   0.0       0.0   
...               ...       ...       ...        ...     ...   ...       ...   
＊ㄴδㅁξ - - ª..     0.0       0.0       0.0        0.0     0.0   0.0       0.0   
＊오가＊              0.0       0.0       0.0        0.0     0.0   0.0       0.0   
＊좋은생각＊            0.0       0.0       0.0        0.0     0.0   0.0       0.0   
￡0ⓥe ⓘs...        0.0       0.0       0.0        0.0     0.0   0.0       0.0   
￡ΟνΕ　 Ιε 。。♡      0.0       0.0       0.